In [64]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlalchemy import create_engine, inspect
import pandas as pd
import json

### (E)TL Extract two datasets

In [66]:
# Extracted CSV Data
primary_df = pd.read_csv('us-2016-primary-results.csv', sep=';')
primary_df.head()

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,Vermont,VT,Sutton,95000197.0,Republican,John Kasich,20,0.227
1,Vermont,VT,Tunbridge,95000204.0,Republican,John Kasich,36,0.319
2,Vermont,VT,Weathersfield,95000220.0,Republican,Ted Cruz,46,0.111
3,Vermont,VT,Weston,95000228.0,Republican,John Kasich,32,0.471
4,Vermont,VT,Whiting,95000233.0,Republican,Ben Carson,0,0.000


In [67]:
# Extracted JSON data
with open('/Users/sahrmusa/Desktop/bootcamp/Homework/ETL-Project/2019_Results_backup.json') as json_file:
    data2 = json.load(json_file)

# df = pd.DataFrame(data2['Andaman & Nicobar Islands'])
# df = pd.DataFrame(data2)

# df.head()
data2

{'Andaman & Nicobar Islands': [{'Constituency': 'Andaman & Nicobar Islands ',
   'O.S.N.': 1,
   'Candidate': 'AYAN MANDAL',
   'Party': 'All India Trinamool Congress',
   'EVM Votes': 1717,
   'Postal Votes': 4,
   'Total Votes': 1721,
   '% of Votes': 0.83,
   'Candidate Won': 'loss'},
  {'Constituency': 'Andaman & Nicobar Islands ',
   'O.S.N.': 2,
   'Candidate': 'KULDEEP RAI SHARMA',
   'Party': 'Indian National Congress',
   'EVM Votes': 95249,
   'Postal Votes': 59,
   'Total Votes': 95308,
   '% of Votes': 45.98,
   'Candidate Won': 'loss'},
  {'Constituency': 'Andaman & Nicobar Islands ',
   'O.S.N.': 3,
   'Candidate': 'PRAKASH MINJ',
   'Party': 'Bahujan Samaj Party',
   'EVM Votes': 2478,
   'Postal Votes': 8,
   'Total Votes': 2486,
   '% of Votes': 1.2,
   'Candidate Won': 'loss'},
  {'Constituency': 'Andaman & Nicobar Islands ',
   'O.S.N.': 4,
   'Candidate': 'VISHAL JOLLY',
   'Party': 'Bharatiya Janata Party',
   'EVM Votes': 93772,
   'Postal Votes': 129,
   'Total V

### E(T)L Transform data

In [68]:
# Add STATE column df table
total_df = pd.DataFrame()
for key in data2.keys():
#     print(key)
    df = pd.DataFrame(data2[key])
    df['State'] = key
    total_df = total_df.append(df)
    
total_df.head()

,Constituency,O.S.N.,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,Candidate Won,State
0,Andaman & Nicobar Islands,1,AYAN MANDAL,All India Trinamool Congress,1717,4,1721,0.83,loss,Andaman & Nicobar Islands
1,Andaman & Nicobar Islands,2,KULDEEP RAI SHARMA,Indian National Congress,95249,59,95308,45.98,loss,Andaman & Nicobar Islands
2,Andaman & Nicobar Islands,3,PRAKASH MINJ,Bahujan Samaj Party,2478,8,2486,1.20,loss,Andaman & Nicobar Islands
3,Andaman & Nicobar Islands,4,VISHAL JOLLY,Bharatiya Janata Party,93772,129,93901,45.30,loss,Andaman & Nicobar Islands
4,Andaman & Nicobar Islands,5,SANJAY MESHACK,Aam Aadmi Party,2837,2,2839,1.37,loss,Andaman & Nicobar Islands


In [69]:
# Drop state_abbreviation, fips, fraction_votes and rename columns
prim_df = primary_df[['state','county','party','candidate','votes']]\
.rename(columns = {'state':'State','county':'Constituency','party':'Party','candidate':'Candidate','votes':'Total Votes'})

# set index to State
prim_df = prim_df.head().set_index('State')
prim_df.head()

,Constituency,Party,Candidate,Total Votes
State,,,,
Vermont,Sutton,Republican,John Kasich,20
Vermont,Tunbridge,Republican,John Kasich,36
Vermont,Weathersfield,Republican,Ted Cruz,46
Vermont,Weston,Republican,John Kasich,32
Vermont,Whiting,Republican,Ben Carson,0


In [70]:
# Drop columns and set index
total_df = total_df[['Constituency','Candidate','Party','Total Votes','State']].set_index('State')

total_df.head()

,Constituency,Candidate,Party,Total Votes
State,,,,
Andaman & Nicobar Islands,Andaman & Nicobar Islands,AYAN MANDAL,All India Trinamool Congress,1721
Andaman & Nicobar Islands,Andaman & Nicobar Islands,KULDEEP RAI SHARMA,Indian National Congress,95308
Andaman & Nicobar Islands,Andaman & Nicobar Islands,PRAKASH MINJ,Bahujan Samaj Party,2486
Andaman & Nicobar Islands,Andaman & Nicobar Islands,VISHAL JOLLY,Bharatiya Janata Party,93901
Andaman & Nicobar Islands,Andaman & Nicobar Islands,SANJAY MESHACK,Aam Aadmi Party,2839


### ET(L) Load

### PostgresSQL Database 

In [65]:
etl_project_db = "postgres:postgres@localhost:5432/ETL_Project_DB"
engine = create_engine(f'postgresql://{etl_project_db}')
# create a session
session = Session(engine)

In [72]:
# upload total_df to database
total_df.to_sql('Total_df', engine)

ValueError: Table 'Total_df' already exists.

In [73]:
primary_df.to_sql('Primary_df',engine)